In [16]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
import time

In [17]:
from models import FFNN

In [18]:
print("Downloading MNIST dataset...")
X, y = fetch_openml('mnist_784', version=1, return_X_y=True, parser='auto')
X = X.astype(float)
y = y.astype(int)

In [19]:
X = np.array(X)
y = np.array(y)

In [20]:
X = X / 255.0

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [22]:
encoder = OneHotEncoder(sparse_output=False, categories='auto')
y_train_one_hot = encoder.fit_transform(y_train.reshape(-1, 1))
y_test_one_hot = encoder.transform(y_test.reshape(-1, 1))

In [23]:
input_size = 784  # images 28x28 = 784 pixels
hidden_sizes = [64] 
output_size = 10  # digits 0-9

In [24]:
ffnn_model = FFNN(
    layer_sizes=[input_size] + hidden_sizes + [output_size],
    activations=['relu'] * len(hidden_sizes) + ['softmax'],
    loss='categorical_cross_entropy',
    weight_init='he'
)

In [25]:
sklearn_model = MLPClassifier(
    hidden_layer_sizes=hidden_sizes,
    activation='relu',
    solver='sgd', 
    learning_rate_init=0.01,
    max_iter=10,
    random_state=42
)

In [26]:
print("\nTraining custom FFNN model...")
start_time = time.time()
ffnn_history = ffnn_model.train(
    X_train, y_train_one_hot,
    batch_size=32,
    learning_rate=0.01,
    epochs=10,
    verbose=1
)
ffnn_training_time = time.time() - start_time
print(f"Custom FFNN training time: {ffnn_training_time:.2f} seconds")


Training custom FFNN model...
Epoch 1/10, Train Loss: 0.7104
Epoch 2/10, Train Loss: 0.3466
Epoch 3/10, Train Loss: 0.2957
Epoch 4/10, Train Loss: 0.2663
Epoch 5/10, Train Loss: 0.2439
Epoch 6/10, Train Loss: 0.2260
Epoch 7/10, Train Loss: 0.2107
Epoch 8/10, Train Loss: 0.1977
Epoch 9/10, Train Loss: 0.1865
Epoch 10/10, Train Loss: 0.1768
Custom FFNN training time: 10.54 seconds


In [27]:
ffnn_predictions = ffnn_model.predict(X_test)
ffnn_pred_classes = np.argmax(ffnn_predictions, axis=1)
ffnn_accuracy = accuracy_score(y_test, ffnn_pred_classes)
ffnn_accuracy

0.9472857142857143

In [28]:
print("\nTraining sklearn MLPClassifier...")
start_time = time.time()
sklearn_model.fit(X_train, y_train)
sklearn_training_time = time.time() - start_time
print(f"sklearn MLPClassifier training time: {sklearn_training_time:.2f} seconds")


Training sklearn MLPClassifier...
sklearn MLPClassifier training time: 4.74 seconds


C:\Users\Lenovo\AppData\Roaming\Python\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10) reached and the optimization hasn't converged yet.
  warnings.warn(


In [29]:
sklearn_predictions = sklearn_model.predict(X_test)
sklearn_accuracy = accuracy_score(y_test, sklearn_predictions)
sklearn_accuracy

0.9530714285714286